## Clase Transaction

Una transacción en el contexto de blockchain es un cambio de estado en la red. Las transacciones de nuestra blockchain permiten que dos usuarios puedan transferirse dinero virtual.

Vamos a crear un archivo en nuestro directorio de trabajo llamado transaction.py. En él, vamos a instanciar una clase de nombre Transaction.

In [1]:
class Transaction():
    pass

Cada transacción tiene 3 elementos principales.
- ¿Quién manda la transaccion? (sender)
- ¿Quién la recibe? (recipient)
- ¿Cuánto dinero es? (value)

Cada que se instancie una transaccion en local, la vamos a inicializar con sus elementos principales.

In [3]:
from bin.account import Account

class Transaction():
    def __init__(self, sender: Account, value: int, recipient: Account):
        pass

Ademas, añadiremos atributos "informativos" que sirven para guardar informacion importante de la transaccion.

- Timestamp: Hora en la que la transaccion fue añadida a la red.
- Block: Bloque al que pertenece la transaccion.
- Signature: Firma digital de la transaccion. 
- Status: Estado actual de la transaccion.

In [67]:
from account import Account 

class Transaction:
    def __init__(self, sender: Account, value: int, recipient: Account):
        self.sender = sender
        self.value = value
        self.recipient = recipient
        # Hora en la que la transaccion se instancia
        self.time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        self.block = None
        self.signature = None
        self.status = TxStatus.PENDIENTE # (1)
        # Al instanciarse una tx, esta debe reflejarse en la cuenta que la envia. (Sender)
        sender.list_of_all_transactions.append(self) # (2)

Del bloque de codigo anterior vamos a resaltar dos puntos:
- (1) Vamos a desarrollar un Enum que nos permita ir haciendo **cambios de estado**.
- (2) Esta linea de codigo anexa en la cuenta del remitente la transaccion que se esta realizando. 

#### Enum

Un Enum nos ayuda a definir nuestros propios tipos de datos. En este caso, vamos a diseñar uno que nos de 3 casos de una transacción:
- Completada
- Rechazada
- En espera

In [68]:
from enum import Enum
class TxStatus(Enum):
    PENDIENTE = 0
    CONFIRMADA = 1
    DECLINADA = 2

El primer metodo de la clase Account es el siguiente:

In [69]:
    def to_dict(self):
        """Exporta la transaccion en formato: dict."""
        return {
            'sender': self.sender.nickname,
            'recipient': self.recipient.nickname,
            'value': self.value,
            'time' : self.time}

Este metodo exporta un diccionario con el "encabezado" de la transaccion. Es la informacion que se codifica y pasa a ser el contenido de la transaccion.

### Firmado y Verificado de transacciones utilizando una cuenta 

#### Firmado

Digamos que nuestra transaccion es lo siguiente:

In [70]:
transaction = {
    "sender": "Pedro",
    "receiver": "maria",
    "amount": 10
}

In [71]:
# Si trataramos de ingresar en un hash este contenido pasaria lo siguiente:
from Crypto.Hash import SHA256
# hash = SHA256.new(transaction) # Sale un errorsaso.

# SHA256 solo acepta valores alfanumericos, por lo tanto necesitamos convertir nuestro diccionario en una cadena de bites
# Primero: dict -> str
transaction_str = str(transaction)

# Ya lo convertimos a String, ahora necesitamos convertirlo a bites
transaction_byte = transaction_str.encode()
type(transaction)

dict

In [72]:
# Una vez lo tenemos en bytes podemos pasarlo por un algoritmo SHA256 sin problema.
tx_hashed = SHA256.new(transaction_byte)

# Imprimimos el hash en su version hexadecimal
tx_hashed.hexdigest()

'6a9519ccc6c7ecececf8801e699fe72544eccb46a37a7387bc416d4f347141a8'

Ya que tenemos el contenido de nuestra transaccion y su hash, podemos obtener su firma.

In [73]:
# Hagamos el ejemplo completo:
# Pedro es el dueno de la transaccion.
# Por lo tanto, Pedro tiene una cuenta con sus respectivas llaves.
from account import Account
pedro = Account('pedro')

# Gracias a que desarrollamos la clase cuenta, podemos utilizarla en este ejemplo.
# Pedro tiene su llave privada y pubica, asi como su firmador y su verificador.
# Pedro procede a firmar la transaccion
signature = pedro.signer.sign(tx_hashed)

# Nuestra firma se ve de la siguiente manera
signature

TypeError: __init__() missing 1 required positional argument: 'nickname'

#### Verificado

Para verificar una transaccion el proceso es muy similar. 

In [ ]:
# Necesitamos pasar por un hash el contenido de la transaccion.
tx_hashed = SHA256.new(str(transaction).encode()) # Todo el hash en una sola linea de codigo

# Ahora, en vez de usar el firmador, vamos a usar el verificador. 
try:
    pedro.verifier.verify(tx_hashed, signature)
    print(True)
except:
    print(False)


El verificador alza un error si la firma es invalida. Por eso utilizamos try/except

In [ ]:
# Si lo intentaramos con una firma falsa el resultado seria distinto
firma_falsa = SHA256.new('transaccion falsa'.encode())
# Ahora, en vez de usar el firmador, vamos a usar el verificador. 
try:
    pedro.verifier.verify(firma_falsa, signature)
    print(True)
except:
    print(False)

Las condiciones para que salga error son:
- El contenido de la transaccion haya sido alterado.
- La firma es incorrecta.
- Quien firma no es el autor de la transaccion.

Vamos a anadir estas funciones a los metodos de nuestra clase Transaction. Nuestro codigo completo quedaria asi:

In [ ]:
from datetime import datetime
from enum import Enum
from account import Account
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Hash import SHA256

class TxStatus(Enum):
    PENDIENTE = 0
    CONFIRMADA = 1
    DECLINADA = 2

class Transaction:
    def __init__(self, sender: Account, value: int, recipient: Account):
        self.sender = sender
        self.value = value
        self.recipient = recipient
        self.time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        self.block = None
        self.signature = None
        self.status = TxStatus.PENDIENTE
        # Al instanciarse una tx, esta debe reflejarse en la cuenta que la envia. (Sender)
        sender.list_of_all_transactions.append(self)


    def to_dict(self):
        """Exporta la transaccion en formato: dict."""
        return {
            'sender': self.sender.nickname,
            'recipient': self.recipient.nickname,
            'value': self.value,
            'time' : self.time}

    def sign_transaction(self): # (1)
        """Funcion que recibe un objeto transaccion y devuelve 
        la firma de la transaccion en bytes"""
        print("Firmando transaccion...")
        msg = str(self.to_dict()).encode()
        hash = SHA256.new(msg)
        signer = self.sender.signer
        signature = signer.sign(hash)
        # print("Signature:", binascii.hexlify(signature))
        self.signature = signature

    def verify_signature(self) -> bool: # (1)
        """Aqui se verifican las transacciones"""
        print("Verificando la firma de la transaccion...")
        msg = str(self.to_dict()).encode()
        hash = SHA256.new(msg)
        verifier = self.sender.verifier
        try:
            verifier.verify(hash, self.signature)
            print("La firma es valida.")
            return True
        except:
            print("La firma es invalida.")
            return False

    def change_status(self, new_status): # (3)
        if new_status == 'CONFIRMADA':
            self.status = TxStatus.CONFIRMADA
        elif new_status == 'PENDIENTE':
            self.status = TxStatus.PENDIENTE
        elif new_status == 'DECLINADA':
            self.status = TxStatus.DECLINADA

Lo que aplicamos se encuentra en las funciones sign_transaction(), verify_signature() y to_dict(). La unica funcion que nos falto comentar es change_status()

#### Funcion change_status()

Funcion que cambia el estado de una transaccion. Hay que recordar que los estados cambian dependiendo del evento, por ejemplo:
- Cuando una transaccion se crea, empieza siendo Pendiente.
- Cuando es parte de la cadena de bloques, pasa a estar confirmada.
- Si la red tiene algun error, podria rechazar la transaccion.